**Klasifikasi Berita Pemilu 2024 Menggunakan Algoritma Decision Tree dan Fitur N-Gram, TF-IDF, serta BOW pada Dataset Detik.com**

In [1]:
%pip install Sastrawi

In [10]:
%pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB 1.4 MB/s eta 0:00:01
     -------------------------------------  41.0/42.0 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 406.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.5 MB 919.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 748.1 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.5 MB 841.6 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.5 MB 793.0 kB/s eta 0:00:02
   ------ --------

In [38]:
import re, string
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# Load Data

In [3]:
path_data = './dataset/news_data.csv'
berita_pemilu = pd.read_csv(path_data)
berita_pemilu=berita_pemilu.dropna() # menghapus data missing value
berita_pemilu.reset_index(inplace=True, drop=True)
berita_pemilu.head()

,title,author,publish_date,article_text,url,main_image,tag
0,Hasto Ingin Debat Pilpres Pertarungan Gagasan ...,detikNews,2023-12-10 23:31:00+07:00,Sekjen PDIP sekaligus Sekretaris Tim Pemenang ...,https://news.detik.com/pemilu/d-7082492/hasto-...,https://awsimages.detik.net.id/community/media...,"['hasto kristiyanto', 'hasto', 'sekjen pdip', ..."
1,Contoh Format dan Isi Surat Pernyataan Anggota...,detikNews,2023-12-10 22:38:00+07:00,Surat pernyataan KPPS Pemilu 2024 merupakan ba...,https://news.detik.com/pemilu/d-7082424/contoh...,https://awsimages.detik.net.id/community/media...,"['surat pernyataan kpps pemilu 2024', 'pendaft..."
2,9 Ribu Personel Gabungan Dikerahkan Amankan Na...,detikSumut,2023-12-10 22:23:00+07:00,Sekitar 9.000 personel dikerahkan untuk mengam...,https://www.detik.com/sumut/berita/d-7082448/9...,https://awsimages.detik.net.id/community/media...,"['natal', 'tahun baru', 'nataru', 'pengamanan ..."
3,Amarah Ukraina Usai Rusia Bakal Gelar Pilpres ...,detikNews,2023-12-10 22:12:00+07:00,Ukraina marah usai Rusia menjadikan empat wila...,https://news.detik.com/internasional/d-7082454...,https://awsimages.detik.net.id/community/media...,"['ukraina', 'rusia', 'pemilu rusia', 'round-up']"
4,TKN Prabowo-Gibran Buka Pintu untuk Eks KSAD D...,detikNews,2023-12-10 21:59:00+07:00,Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,https://news.detik.com/pemilu/d-7082445/tkn-pr...,https://awsimages.detik.net.id/community/media...,"['tkn', 'prabowo-gibran', 'dudung abdurachman']"


# Pre-Processing & Labeling

Pre-processing pada data text yang dilakukan:
- Data cleaning
- Lowercasing
- Tokenizer
- Stopword, sekaligus pembuatan corpus/vocabulary kata

Kemudian dilakukan labeling yang berfungsi untuk mengelompokan berita yang ada sesuai dengan keyword yang ada pada berita terdiri dari 3 label yaitu paslon_1, paslon_2, paslon_3.

Kemudian berita yang sudah dilabeling dilakukan encoder agar dapat dibaca oleh mesin.

In [4]:
def preprocess_text(text):
    '''Fungsi preprocess_text membersihkan dan siapkan teks paragraf untuk tokenisasi'''
    if not isinstance(text, str):
        return text

    lower_case = text.lower()
    hasil = re.sub(r'[,()\d]+', '', lower_case) # Menghapus angka, tanda kurung, dan koma
    hasil = re.sub(r'#\w+|@\w+', '', hasil) # Menghapus hashtag dan mention
    hasil = re.sub('<.*?>', ' ', hasil) # Menghapus karakter HTML
    hasil = re.sub('[^a-zA-Z0-9]', ' ', hasil) # Mempertimbangkan huruf dan angka
    hasil = re.sub("\n", " ", hasil)  # Mengganti line baru dengan spasi
    hasil = re.sub(r"\b[a-zA-Z]\b", " ", hasil) # Menghapus single char
    hasil = ' '.join(hasil.split()) # Memisahkan dan menggabungkan kata
    return hasil


In [5]:
df=berita_pemilu[['title', 'article_text']].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17935 entries, 0 to 17934
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         17935 non-null  object
 1   article_text  17935 non-null  object
dtypes: object(2)
memory usage: 280.4+ KB


In [6]:
df['merge_title_article'] = df['title'] + ' ' + df['article_text']

# preses merge dataset
df['news'] = df['merge_title_article'].apply(preprocess_text)
df.drop(columns=['merge_title_article'], inplace=True)
df.head()

,title,article_text,news
0,Hasto Ingin Debat Pilpres Pertarungan Gagasan ...,Sekjen PDIP sekaligus Sekretaris Tim Pemenang ...,hasto ingin debat pilpres pertarungan gagasan ...
1,Contoh Format dan Isi Surat Pernyataan Anggota...,Surat pernyataan KPPS Pemilu 2024 merupakan ba...,contoh format dan isi surat pernyataan anggota...
2,9 Ribu Personel Gabungan Dikerahkan Amankan Na...,Sekitar 9.000 personel dikerahkan untuk mengam...,ribu personel gabungan dikerahkan amankan nata...
3,Amarah Ukraina Usai Rusia Bakal Gelar Pilpres ...,Ukraina marah usai Rusia menjadikan empat wila...,amarah ukraina usai rusia bakal gelar pilpres ...
4,TKN Prabowo-Gibran Buka Pintu untuk Eks KSAD D...,Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,tkn prabowo gibran buka pintu untuk eks ksad d...


In [7]:
tokenized = df['news'].apply(lambda x: x.split())

tokenized_df = pd.DataFrame({'Teks News': df['news'], 'Tokenized': tokenized})
tokenized_df

,Teks News,Tokenized
0,hasto ingin debat pilpres pertarungan gagasan ...,"[hasto, ingin, debat, pilpres, pertarungan, ga..."
1,contoh format dan isi surat pernyataan anggota...,"[contoh, format, dan, isi, surat, pernyataan, ..."
2,ribu personel gabungan dikerahkan amankan nata...,"[ribu, personel, gabungan, dikerahkan, amankan..."
3,amarah ukraina usai rusia bakal gelar pilpres ...,"[amarah, ukraina, usai, rusia, bakal, gelar, p..."
4,tkn prabowo gibran buka pintu untuk eks ksad d...,"[tkn, prabowo, gibran, buka, pintu, untuk, eks..."
...,...,...
17930,alkisah romahurmuziy kembali mesra dengan part...,"[alkisah, romahurmuziy, kembali, mesra, dengan..."
17931,pks harap ri terhindar dari resesi di tak ada ...,"[pks, harap, ri, terhindar, dari, resesi, di, ..."
17932,resolusi pkb di cak imin maju capres hingga ku...,"[resolusi, pkb, di, cak, imin, maju, capres, h..."
17933,menerka antara puan atau ganjar capres yang ak...,"[menerka, antara, puan, atau, ganjar, capres, ..."


In [8]:
#Stopword
import Sastrawi

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text


df['news'] = df['news'].apply(lambda x: stopword(x))
df

,title,article_text,news
0,Hasto Ingin Debat Pilpres Pertarungan Gagasan ...,Sekjen PDIP sekaligus Sekretaris Tim Pemenang ...,hasto debat pilpres pertarungan gagasan tak re...
1,Contoh Format dan Isi Surat Pernyataan Anggota...,Surat pernyataan KPPS Pemilu 2024 merupakan ba...,contoh format isi surat pernyataan anggota kpp...
2,9 Ribu Personel Gabungan Dikerahkan Amankan Na...,Sekitar 9.000 personel dikerahkan untuk mengam...,ribu personel gabungan dikerahkan amankan nata...
3,Amarah Ukraina Usai Rusia Bakal Gelar Pilpres ...,Ukraina marah usai Rusia menjadikan empat wila...,amarah ukraina usai rusia bakal gelar pilpres ...
4,TKN Prabowo-Gibran Buka Pintu untuk Eks KSAD D...,Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,tkn prabowo gibran buka pintu eks ksad dudung ...
...,...,...,...
17930,Alkisah Romahurmuziy Kembali Mesra dengan Part...,Sekonyong-konyong eks Ketum PPP Romahurmuziy m...,alkisah romahurmuziy mesra partai kakbah sekon...
17931,"PKS Harap RI Terhindar dari Resesi di 2023, Ta...",Partai Keadilan Sejahtera (PKS) menyampaikan r...,pks harap ri terhindar resesi tak penundaan pe...
17932,Resolusi PKB di 2023: Cak Imin Maju Capres hin...,Indonesia sudah memasuki Tahun Baru 2023. Semu...,resolusi pkb cak imin maju capres hingga kursi...
17933,Menerka Antara Puan atau Ganjar Capres yang Ak...,Ketua Umum PDIP Megawati Soekarnoputri disebut...,menerka puan ganjar capres diumumkan mega ketu...


In [11]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    print(i)
    title = re.sub('[^a-zA-Z]', ' ', df['news'][i])
    title = title.lower()
    title = title.split()
    title = ' '.join(title)
    corpus.append(title)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
corpus

['hasto debat pilpres pertarungan gagasan tak rekayasa sekjen pdip sekaligus sekretaris tim pemenang nasional tpn ganjar pranowo mahfud md hasto kristiyanto debat calon presiden calon wakil presiden saling adu gagasan menyebut debat pertarungan gagasan rekayasa debat betul betul menjadi pertarungan gagasan otentik pertarungan gagasan tidak suatu rekayasa kata hasto wartawan serang banten minggu hasro capres cawapres menunjukkan jati dirinya pemimpin otentik bisa menampilkan visi misi programnya pemimpin advertisement scroll to continue with content bukan mendompleng siapa ucapnya makanya mengatakan ganjar mahfud siap rangkaian debat diselenggarakan kpu debat tpn pertarungan gagasan disaksikan rakyat indonesia ganjar mahfud siap melakukan debat tegasnya samping tpn setuju para panelis debat membuat pakta integritas tidak membocorkan pertanyaan debat tpn melihat seluruh tahapan pemilu rentan intimidasi menurutnya kelompok khawatir gerakan rakyat ingin pemimpin baik intimidasi menunjukkan

**Labeling**

Mengelompokan berita yang ada sesuai dengan keyword yang ada pada berita terdiri dari 3 label yaitu paslon_1, paslon_2, paslon_3.

In [13]:
def labeling_text(text, label_map):
  group_word = {key: [] for key in label_map.keys()}
  for word in text.split():
      for label, keywords in label_map.items():
          # Memeriksa apakah kata secara eksak cocok dengan salah satu keyword
          if word in keywords:
              group_word[label].append(word)

  # muncul setidaknya 10x dalam 1 dataset
  valid_data = {k: v for k, v in group_word.items() if len(v) > 10}
  if not valid_data:
        return np.nan

  label = max(valid_data, key=lambda k: len(valid_data[k])) if any(valid_data.values()) else np.nan
  return label

label_map={
    'paslon_1':[
        'anies', 'baswedan', 'anies baswedan',
        'amin', 'muhaimin','iskandar', 'muhaimin iskandar', 'cak imin',
        'anies muhaimin'
    ],
    'paslon_2':[
        'prabowo', 'subianto', 'prabowo subianto',
        'gibran', 'rakabuming', 'gibran rakabuming',
        'prabowo gibran'
    ],
    'paslon_3':[
        'ganjar', 'pranowo','ganjar pranowo',
        'mahfud',
        'ganjar mahfud'
    ]}


In [14]:
df['label'] = df['news'].apply(lambda x: labeling_text(x, label_map))
df.head()

,title,article_text,news,label
0,Hasto Ingin Debat Pilpres Pertarungan Gagasan ...,Sekjen PDIP sekaligus Sekretaris Tim Pemenang ...,hasto debat pilpres pertarungan gagasan tak re...,NaN
1,Contoh Format dan Isi Surat Pernyataan Anggota...,Surat pernyataan KPPS Pemilu 2024 merupakan ba...,contoh format isi surat pernyataan anggota kpp...,NaN
2,9 Ribu Personel Gabungan Dikerahkan Amankan Na...,Sekitar 9.000 personel dikerahkan untuk mengam...,ribu personel gabungan dikerahkan amankan nata...,NaN
3,Amarah Ukraina Usai Rusia Bakal Gelar Pilpres ...,Ukraina marah usai Rusia menjadikan empat wila...,amarah ukraina usai rusia bakal gelar pilpres ...,NaN
4,TKN Prabowo-Gibran Buka Pintu untuk Eks KSAD D...,Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,tkn prabowo gibran buka pintu eks ksad dudung ...,NaN


In [15]:
df=df.dropna() # menghapus data missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7127 entries, 5 to 17933
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         7127 non-null   object
 1   article_text  7127 non-null   object
 2   news          7127 non-null   object
 3   label         7127 non-null   object
dtypes: object(4)
memory usage: 278.4+ KB


In [47]:
# Jumlah berita berdasarkan paslon
df.groupby('label')['news'].count().reset_index(name='Topik Berita')

,label,Topik Berita
0,paslon_1,2045
1,paslon_2,3105
2,paslon_3,1977


In [48]:
df_model = df[['news', 'label']]
df_model.reset_index(inplace=True, drop=True)
category = pd.get_dummies(df_model.label).astype(int)

df_model=pd.concat([df_model, category], axis=1)
df_model = df_model.drop(columns='label')
df_model.head()

,news,paslon_1,paslon_2,paslon_3
0,gibran dadakan makan gultik blok malam traktir...,0,1,0
1,tpn ganjar mahfud anggap dukungan abuya muhtad...,0,0,1
2,tkn prabowo marapi menhan wajar naik helikopte...,0,1,0
3,prabowo gibran siap hadapi debat pertama pilpr...,0,1,0
4,debat perdana pilpres tpd jabar ganjar mahfud ...,0,0,1


In [49]:
df_model = df[['news', 'label']]
df_model.reset_index(inplace=True, drop=True)
category = pd.get_dummies(df_model.label).astype(int)

df_model=pd.concat([df_model, category], axis=1)
df_model = df_model.drop(columns='label')
df_model.head()

,news,paslon_1,paslon_2,paslon_3
0,gibran dadakan makan gultik blok malam traktir...,0,1,0
1,tpn ganjar mahfud anggap dukungan abuya muhtad...,0,0,1
2,tkn prabowo marapi menhan wajar naik helikopte...,0,1,0
3,prabowo gibran siap hadapi debat pertama pilpr...,0,1,0
4,debat perdana pilpres tpd jabar ganjar mahfud ...,0,0,1


In [50]:
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# df_model['label_encoded'] = label_encoder.fit_transform(df_model['label'])
# df_model

# TF-IDF

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [52]:
vectorizer_tfidf = TfidfVectorizer(decode_error='replace', encoding='utf-8')

X = vectorizer_tfidf.fit_transform(df_model.news)
Y = df_model['label_encoded']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

model_TF = DecisionTreeClassifier(random_state=42)
model_TF.fit(x_train, y_train)

y_pred_TF = model_TF.predict(x_test)

# Print classification report
print(classification_report(y_test, y_pred_TF))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_TF))

KeyError: 'label_encoded'

# Deep Learning

In [53]:
news = df_model['news'].values
label = df_model[['paslon_1', 'paslon_2', 'paslon_3']].values

news_train, news_test, label_train, label_test = train_test_split(news, label, test_size=0.2)

In [54]:
# @title Tokenizer dataset

tokenizer = Tokenizer(num_words=20000, oov_token='x')
tokenizer.fit_on_texts(news_train)
tokenizer.fit_on_texts(news_test)

sekuens_train = tokenizer.texts_to_sequences(news_train)
sekuens_test = tokenizer.texts_to_sequences(news_test)

padded_train = pad_sequences(sekuens_train, maxlen=8958)
padded_test = pad_sequences(sekuens_test, maxlen=8958)

In [55]:
import pickle
# Simpan tokenizer ke dalam file
with open("./tokenizer.pkl", 'wb') as f:
    pickle.dump(tokenizer, f)

padded_train.shape

(5701, 8958)

In [56]:
# @title Callbacks

class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > 0.93:
            print("\nAkurasi telah mencapai >93%!")
            self.model.stop_training = True

callbacks = MyCallback()

In [57]:
# @title CNN Arsitektur

model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=100, input_length=8958))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

c:\Users\Tazkiiia\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [58]:
# @title Train Models

num_epochs = 30
batch_size = 100

history = model.fit(
    padded_train,
    label_train,
    epochs=num_epochs,
    validation_data=(padded_test, label_test),
    verbose=1,
    batch_size=batch_size,
    callbacks=[
            callbacks,
        ],
)

Epoch 1/30
40/58 ━━━━━━━━━━━━━━━━━━━━ 1:16:32 255s/step - accuracy: 0.4183 - loss: 1.0855

In [ ]:
model.save('model.h5')

In [ ]:
# @title Test model menggunakan 10 data pertama dari df_model sebagai data sample

# Fungsi untuk melakukan prediksi pada data baru
def predict_data_sample(model, tokenizer, data_sample):
    sequences_data_sample = tokenizer.texts_to_sequences(data_sample)
    padded_data_sample = pad_sequences(sequences_data_sample, maxlen=8958)
    predictions = model.predict(padded_data_sample)
    predicted_labels = np.argmax(predictions, axis=1) +1
    return [f'paslon_{num}' for num in predicted_labels]

tokenizer_path = './tokenizer.pkl'
with open(tokenizer_path, 'rb') as token:
    tokenizer = pickle.load(token)

In [ ]:
# mengambil sample 10 data pertama
df_sample = df_model.iloc[:10].copy()

original=df_sample[['paslon_1', 'paslon_2', 'paslon_3']].idxmax(axis=1)
sample = df_sample['news'].tolist()

data_predict = predict_data_sample(model, tokenizer, sample) # Melakukan prediksi

for ori, pred in zip(original, data_predict):
  print(f"Original: {ori}, Predict: {pred}, ==> {'Benar' if ori==pred else 'Salah'}")

# LSTM

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SpatialDropout1D, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Assuming df_model is already defined

# Tokenizer and padding for LSTM
max_words = 10000
max_len = 50
embedding_dim = 32

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_model.news)
X = tokenizer.texts_to_sequences(df_model.news)
X = pad_sequences(X, maxlen=max_len)

Y = df_model['label_encoded']

# Splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Build LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(max_words, embedding_dim, input_length=max_len))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(3, activation='softmax'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
print(model_lstm.summary())

# Training the model
model_lstm.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Predictions and evaluation
y_pred_lstm = np.argmax(model_lstm.predict(x_test), axis=-1)
print(classification_report(y_test, y_pred_lstm))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lstm))


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
156/156 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.5340 - loss: 0.9081 - val_accuracy: 0.8855 - val_loss: 0.3271
Epoch 2/5
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.8906 - loss: 0.3627 - val_accuracy: 0.8981 - val_loss: 0.3242
Epoch 3/5
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.9302 - loss: 0.2544 - val_accuracy: 0.9023 - val_loss: 0.2786
Epoch 4/5
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.9439 - loss: 0.1845 - val_accuracy: 0.8794 - val_loss: 0.3928
Epoch 5/5
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.9573 - loss: 0.1487 - val_accuracy: 0.8976 - val_loss: 0.3078
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       610
           1       0.90      0.92      0.91       933
           2       0.90      0.85      0.88       596

    accuracy                           0.90      2139
   macro avg       0.90      0.89      0.8